In [80]:
import re
import os
import pandas as pd
import pdfplumber

In [81]:
pdf_path = "/Users/shivangsingh/Downloads/2.15-3 PFCM 2020-1 Servicer Report - Feb 25 (UK EU).pdf"
folder_name = "dfc_files"

In [82]:
with pdfplumber.open(pdf_path) as pdf:
    page = pdf.pages[0]  # Page index 1 for Page 2
    text = page.extract_text()  # Extract text instead of table

lines = text.split("\n")

for line in lines:
    print(line)


PFCM 2020-1 Servicer Report - Feb 25 (UK EU)
COLLATERAL STRATS
Feb Loan Tape
1 Collateral Strats
All balances principal, admin fee and grace interest USD Loans
unless stated otherwise USD
# Loans 9,124
Principal Balance & Admin Fee Only 9 0,168,991
Plus Interest During Grace 9 9,252,731
Average Loan Size 10,878
Current Balance 103,595,247
Average Loan Size 11,354
Loan Approved Balance & Admin Fee 322,995,009
Average Loan Size 35,401
Loans In Repayment Stage Balance 9 5,645,151
MBA Course Balance 2 1,241,503
Non-MBA Course Balance 8,921,736
Non-Business Course Balance 2 5,901,808
Female Borrower Balance 3 6,391,241
Low and Lower Middle-Income Countries Balance 8 1,676,701
Permitted Modification Loan Balance 923,890
WA Margin Loans (%) 8.16%
Lowest Margin (%) 3.50%
Last Maturity Date 15/03/2045
WA Original Repay Period (Years) 1 5.0
Grace Loans WA Remaining Grace Period (Months) 3.7
2 Delinquency & Payment Arrangement Data
All balances principal, admin fee and grace interest USD Loans # 

In [83]:

key_value_pairs = [
    (lines[1], ""),
    (lines[2], "")
]

# Convert the key-value pairs into a DataFrame
df1 = pd.DataFrame(key_value_pairs, columns=["Key", "Value"])
df1
output_filename = os.path.join(folder_name, 'extracted_df1.xlsx')
df1.to_excel(output_filename, index=False, header=False)

In [84]:
def is_number(s):
    """Check if a string represents a number (integer or float)."""
    try:
        float(s)  # Try converting to float
        return True
    except ValueError:
        return False

def process_strings_limited(s):
    s = s.replace(",", "")  # Remove all commas
    words = s.split()

    # Check if the last two words are numbers (integer or float)
    if len(words) >= 2 and is_number(words[-1]) and is_number(words[-2]):
        words[-2] = words[-2] + words[-1]  # Concatenate numbers
        words.pop()  # Remove the last word
    
    return " ".join(words)

cleaned_lines = [process_strings_limited(line) for line in lines[4:26]]
cleaned_lines

['All balances principal admin fee and grace interest USD Loans',
 'unless stated otherwise USD',
 '# Loans 9124',
 'Principal Balance & Admin Fee Only 90168991',
 'Plus Interest During Grace 99252731',
 'Average Loan Size 10878',
 'Current Balance 103595247',
 'Average Loan Size 11354',
 'Loan Approved Balance & Admin Fee 322995009',
 'Average Loan Size 35401',
 'Loans In Repayment Stage Balance 95645151',
 'MBA Course Balance 21241503',
 'Non-MBA Course Balance 8921736',
 'Non-Business Course Balance 25901808',
 'Female Borrower Balance 36391241',
 'Low and Lower Middle-Income Countries Balance 81676701',
 'Permitted Modification Loan Balance 923890',
 'WA Margin Loans (%) 8.16%',
 'Lowest Margin (%) 3.50%',
 'Last Maturity Date 15/03/2045',
 'WA Original Repay Period (Years) 15.0',
 'Grace Loans WA Remaining Grace Period (Months) 3.7']

In [85]:
data = []
for line in cleaned_lines:
    words = line.split()
    key = " ".join(words[:-1])  # Everything except last word
    value = words[-1]  # Last word is the value
    data.append((key, value))

df2 = pd.DataFrame(data, columns=[lines[3], ""])
# df2
output_filename = os.path.join(folder_name, 'extracted_df2.xlsx')
df2.to_excel(output_filename, index=False, header=False)

In [86]:

def clean_number_format(line):
    # Remove commas
    line = line.replace(",", "")
    
    # Split the line into words
    parts = line.split()
    
    # Ensure there are at least three elements to check
    if len(parts) > 3:
        # Check if the last three parts are numbers (including decimals)
        if re.match(r'^\d+(\.\d+)?$', parts[-1]) and re.match(r'^\d+(\.\d+)?$', parts[-2]) and re.match(r'^\d+(\.\d+)?$', parts[-3]):
            # Concatenate -3 and -2
            parts[-3] = parts[-3] + parts[-2]
            parts.pop(-2)  # Remove the now redundant -2
    
    return " ".join(parts)

cleaned_lines = [clean_number_format(line) for line in lines[26:-2]]
cleaned_lines


['2 Delinquency & Payment Arrangement Data',
 'All balances principal admin fee and grace interest USD Loans # Loans',
 'unless stated otherwise USD',
 'Delinquency',
 'a Current 69723696 8281',
 'b Delinquent <=30 days 5097609 151',
 'c Delinquent 31-60 days 3347471 103',
 'd Delinquent 61-90 days 2503527 73',
 'e Delinquent 91-120 days 2170116 61',
 'f Delinquent 121 - 150 days 1025403 30',
 'g Delinquent 151 - 180 days 726597 21',
 'h Delinquent 180+ days (Default) 14658313 404',
 'Payment Arrangement',
 'g Non Study Extension 4348227 126']

In [87]:
data = []
for line in cleaned_lines:
    parts = line.split()
    if len(parts) >= 3 and parts[-1].isdigit() and parts[-2].isdigit():
        name = " ".join(parts[:-2])  # Everything except last two parts
        num1 = parts[-2]  # Second last part
        num2 = parts[-1]  # Last part
        data.append([name, num1, num2])
    else:
        data.append([line, "", ""])  # Keep the first column, leave the others empty

# Create DataFrame
df3 = pd.DataFrame(data, columns=["", "", ""])
# df3
output_filename = os.path.join(folder_name, 'extracted_df3.xlsx')
df3.to_excel(output_filename, index=False, header=False)

In [88]:
# Page 2
with pdfplumber.open(pdf_path) as pdf:
    page_5 = pdf.pages[4]  # Page index 1 for Page 2
    text = page_5.extract_text()  # Extract text instead of table

lines = text.split("\n")
lines

['PFCM 2020-1 Servicer Report - Feb 25 (UK EU)',
 'SERVICER INFORMATION',
 '1 Dates',
 'Report Delivery Date 19/02/20258 Business Days after Cut off Date',
 'Period Start Date 07/01/2025 Current Report Cut-Off 7 Feb 25',
 'Period Cut Off Date 07/02/2025 Previous Report Cut-Off 7 Jan 25',
 'Next Interest Payment Date 07/03/2025 Next Report Cut-Off 7 Mar 25',
 '2 Reporting Period SOFR Information Fixes 7th JAJO',
 '3M USD SOFR Fix Date 07/01/2025 30 Day Average SOFR Fix Date (monthly) 07/01/2025',
 '3M USD SOFR Rate 4.28516% 30 Day Average SOFR Rate 4.47178%',
 '3 Period Collections',
 'USD',
 'Grace Interest Collections 3 59,344.34',
 'Post Grace Interest Collections 9 07,799.80',
 'Arrears Interest Collections 7 7.94',
 'Total Interest Receipts 1,267,222.08',
 'Principal Collections 2,417,313.63',
 'Admin Fee Collections 70,943.71',
 'Total Principal Receipts 2,488,257.34',
 'Legal Fee Proceeds Recovered from Obligors',
 'Legal Fee Proceeds applied to Available Proceeds on 7 Mar 25 2,7

In [89]:
index_4 = next((i for i, line in enumerate(lines) if line.startswith("4")), -1)
index_4

26

In [90]:

def clean_number_format(line):
    # Remove commas
    line = line.replace(",", "")

    # Split the line into words
    parts = line.split()

    # Iterate through the words and merge numbers separated by spaces
    cleaned_parts = []
    i = 0
    while i < len(parts):
        # Check if current and next part are numbers
        if i < len(parts) - 1 and re.match(r'^\d+$', parts[i]) and re.match(r'^\d+$', parts[i+1]):
            cleaned_parts.append(parts[i] + parts[i+1])  # Merge numbers
            i += 2  # Skip next part as it's merged
        else:
            cleaned_parts.append(parts[i])
            i += 1

    return " ".join(cleaned_parts)

# Example usage
cleaned_lines = [clean_number_format(line) for line in lines[26:-1]]
cleaned_lines


['4 Prodigy Financial Covenants',
 'USD Eq',
 'Test Date 07/02/2025',
 'Minimum Tangible Net Worth Covenant',
 'Paid up issued share capital of PIL and dist & non dist reserves 171365491',
 'Plus convertible loan notes -',
 'Less debit balance on PnL (151512220)',
 'Excluding increase in asset valuation after most recent audited financials -',
 'Excluding goodwill and intangible assets (303457)',
 'Excluding capitalised transaction expenses 112690',
 'Excluding mark-to-market movements in any hedging agreements -',
 'Tangible Net Worth 19662504',
 'Minimum Tangible Net Worth Requirement 5000000',
 'Tangible Net Worth Covenant Test Pass/Fail PASS',
 'Minimum Cash & Cash Equivalents Covenant',
 'PIL Group Cash & Cash Equivalents 15250035',
 'Minimum Cash and Cash Equivalents Requirement 5000000',
 'Cash Covenant Test Pass/Fail PASS',
 '6 month burn rate',
 'Total income 18903122',
 '(Increase) / Decrease in trade and other receivables 478744',
 'Less: non-recurring revenues -',
 'A: Oper

In [91]:

def split_key_value(line):
    parts = line.rsplit(" ", 1)  # Split at the last space
    last_part = parts[-1].strip()

    # Check if last part is a number (including negative numbers in parentheses) or PASS/FAIL
    if re.match(r'^-?\d+$', last_part) or re.match(r'^\(-?\d+\)$', last_part) or re.match(r'^\d{2}/\d{2}/\d{4}$', last_part) or last_part in {"PASS", "FAIL", "-"}:
        key = parts[0] if len(parts) > 1 else ""
        value = last_part
    else:
        key = line  # Whole line as key
        value = ""

    return key, value

data = [split_key_value(line) for line in cleaned_lines]
df4 = pd.DataFrame(data, columns=["", ""])
# df4
output_filename = os.path.join(folder_name, 'extracted_df4.xlsx')
df4.to_excel(output_filename, index=False, header=False)

In [92]:
# Page 6
with pdfplumber.open(pdf_path) as pdf:
    page_5 = pdf.pages[5]  # Page index 1 for Page 2
    text = page_5.extract_text()  # Extract text instead of table

lines = text.split("\n")
lines

['PFCM 2020-1 Servicer Report - Feb 25 (UK EU)',
 '5 Performance Triggers',
 'Test Date 07/02/2025',
 'Test Loan Portfolio Purchased',
 'USD',
 'Portfolio Cumulative Default Ratio',
 'Cum. Default Rate 4.23%',
 'Cum. Default Trigger Level 11.00%',
 'Cum. Default Rate Test Pass/Fail PASS',
 'Arrangement Ratio (Long)',
 'Arranagement Ratio 1.77%',
 'Forbearance Ratio Trigger Level 5.00%',
 'Forbearance Ratio Test Pass/Fail PASS',
 'Ratio Numerator 1,766,724',
 'Ratio Denominator 99,563,532',
 'Arrangement Ratio (Short)',
 'Arranagement Ratio 2.79%',
 'Forbearance Ratio Trigger Level 12.00%',
 'Forbearance Ratio Test Pass/Fail PASS',
 'Ratio Numerator 2,779,456',
 'Ratio Denominator 99,563,532',
 'Weighted Average Margin',
 'Weighted Average Interest Margin 8.16%',
 'Weighted Average Interest Margin Trigger Level (Min) 5.25%',
 'Weighted Average Margin Test Pass/Fail PASS',
 'In-Study Forbearance Loans',
 'Count of loans in In-Study Forbearance 15.00',
 'In-Study Forbearance Loans Trigger

In [93]:
index_4 = next((i for i, line in enumerate(lines) if line.startswith("6")), -1)
index_4

34

In [94]:

def clean_number_format(line):
    # Remove commas
    line = line.replace(",", "")

    # Split the line into words
    parts = line.split()

    # Iterate through the words and merge numbers separated by spaces
    cleaned_parts = []
    i = 0
    while i < len(parts):
        # Check if current and next part are numbers
        if i < len(parts) - 1 and re.match(r'^\d+$', parts[i]) and re.match(r'^\d+$', parts[i+1]):
            cleaned_parts.append(parts[i] + parts[i+1])  # Merge numbers
            i += 2  # Skip next part as it's merged
        else:
            cleaned_parts.append(parts[i])
            i += 1

    return " ".join(cleaned_parts)

# Example usage
cleaned_lines = [clean_number_format(line) for line in lines[3:34]]
cleaned_lines


['Test Loan Portfolio Purchased',
 'USD',
 'Portfolio Cumulative Default Ratio',
 'Cum. Default Rate 4.23%',
 'Cum. Default Trigger Level 11.00%',
 'Cum. Default Rate Test Pass/Fail PASS',
 'Arrangement Ratio (Long)',
 'Arranagement Ratio 1.77%',
 'Forbearance Ratio Trigger Level 5.00%',
 'Forbearance Ratio Test Pass/Fail PASS',
 'Ratio Numerator 1766724',
 'Ratio Denominator 99563532',
 'Arrangement Ratio (Short)',
 'Arranagement Ratio 2.79%',
 'Forbearance Ratio Trigger Level 12.00%',
 'Forbearance Ratio Test Pass/Fail PASS',
 'Ratio Numerator 2779456',
 'Ratio Denominator 99563532',
 'Weighted Average Margin',
 'Weighted Average Interest Margin 8.16%',
 'Weighted Average Interest Margin Trigger Level (Min) 5.25%',
 'Weighted Average Margin Test Pass/Fail PASS',
 'In-Study Forbearance Loans',
 'Count of loans in In-Study Forbearance 15.00',
 'In-Study Forbearance Loans Trigger Level 75.00',
 'In-Study Forbearance Loans Test Pass/Fail PASS',
 'Other Triggers Tested on All Receivables'

In [95]:
def split_key_value(line):
    parts = line.rsplit(" ", 1)  # Split at the last space
    last_part = parts[-1].strip()

    # Check if last part is a number (including negative numbers in parentheses) or PASS/FAIL
    if  re.match(r'^-?\d+$', last_part) or re.match(r'^\(-?\d+\)$', last_part) or re.match(r'^\d{2}/\d{2}/\d{4}$', last_part) or re.match(r'^-?\d+(\.\d+)?%$', last_part) or re.match(r'^-?\d+\.\d+$', last_part) or last_part in {"PASS", "FAIL", "-"}:
        key = parts[0] if len(parts) > 1 else ""
        value = last_part
    else:
        key = line  # Whole line as key
        value = ""

    return key, value

data = [split_key_value(line) for line in cleaned_lines]
df5 = pd.DataFrame(data, columns=["", ""])
# df5
output_filename = os.path.join(folder_name, 'extracted_df5.xlsx')
df5.to_excel(output_filename, index=False, header=False)

In [96]:
output_file = 'prodigy_dfc_pdf_output.xlsx'
tables_folder = os.path.join(os.getcwd(), "dfc_files")
excel_files = [f for f in os.listdir(tables_folder) if f.endswith('.xlsx')]
sorted_file_names = sorted(excel_files, key=lambda x: int(re.search(r'(\d+)', x).group()))
# sorted_file_names
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    start_row = 0  # Track the starting row for each dataset
    
    for file in sorted_file_names:
        # Read the data from the current file
        file_path = os.path.join(folder_name, file)
        # print("file_path", file_path)
        data = pd.read_excel(file_path)
        
        # Write data to the final Excel file at the correct row position
        data.to_excel(writer, sheet_name='Sheet1', startrow=start_row, index=False)

        # Update start_row to place the next dataset after 2 empty rows
        start_row += len(data) + 2 